# KFold cross-validation for regression and classification

In [ ]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_boston
from sklearn.linear_model import Ridge
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import mean_squared_error

## Exercise 1: Model selection for regression

### Question 1: load dataset

In [ ]:
# Load Boston dataset:
boston = load_boston()
X = boston.data
Y = boston.target
n_samples, n_features = X.shape

### Question 2: Investigate dataset

In [ ]:
# Scatterplots of target prices with respect to each of the 13 features:
fig, axes = plt.subplots(5,3, figsize=(15, 12), tight_layout=True)
for i, ax in enumerate(axes.flat[:n_features]) :
    ax.scatter(X[:,i], Y, marker='o')
    ax.set_xlabel(boston.feature_names[i])
    ax.set_ylabel('Target price')
fig.suptitle("Target prices with respect to each of the 13 features")
plt.show()

### Question 3 Extract 2 relevant and continuous features

The target prices appear to be strongly correlated to several continuous features... among them, features 5 ("RM") and 12 ("LSTAT").

In [ ]:
fig, ax = plt.subplots()
ax.scatter(X[:,5], X[:,12])
ax.set_title("Visualizing correlation of features 5 & 12")
ax.set_xlabel(boston.feature_names[5])
ax.set_ylabel(boston.feature_names[12])
plt.show()

In [ ]:
# Keep only features 5 and 12
X = X[:,[5,12]]

### Question 6: Simple machine learning pipeline (yes I do question 6 before 4 and 5 :)... )

Here we define functions implementing the basic building blocks of a very simple machine learning pipeline as well as the pipeline itself, which includes:

1. dataset split into 3 datasets training, validation and test
2. training
3. model selection
4. model evaluation

In [ ]:
def split_dataset(X, Y, test_ratio=0.2, val_ratio=0.2, seed=264):
    """
    Split dataset into training, validation, test datasets
    """
    # Extract test set from entire dataset
    X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=test_ratio, shuffle=True, random_state=seed)
    # Extract validation set from train_val dataset
    X_train, X_val, Y_train, Y_val = train_test_split(X_train_val, Y_train_val, test_size=val_ratio, shuffle=True, random_state=seed)
    
    print("Training dataset size:   ", len(X_train))
    print("Validation dataset size: ", len(X_val))
    print("Test dataset size:       ", len(X_test))
    return X_train, Y_train, X_val, Y_val, X_test, Y_test

def polynomial_model(degree, regularization, X_train, X_val, X_test):
    """
    Instantiate a Ridge model and transform data into polynomial features
    """
    # Build the polynomial features:
    poly_features = PolynomialFeatures(degree=degree)
    X_train_poly = poly_features.fit_transform(X_train)
    X_val_poly = poly_features.transform(X_val)
    X_test_poly = poly_features.transform(X_test)
    # Instantiate a Ridge model:
    model = Ridge(alpha=regularization)
    return model, X_train_poly, X_val_poly, X_test_poly

def evaluate(model, X, Y, metric):
    """
    Evaluate (already trained) model performance on the given dataset 
    
    'metric' is performance metric, a python function comparing 'Y' and 'Y_pred'
    """
    Y_pred = model.predict(X)
    perf = metric(Y, Y_pred)
    return perf
    
def model_selection(hparams, val_perf, minimize):
    """
    Return index of the best model given their validation performances
    
    'hparams' and 'val_perf' have the same number of elements
    
    Note: Some performance metrics have to be minimized (e.g. MSE) and 
    some have to be maximized (e.g. accuracy, f1-score)
    """
    if minimize:
        i_best = np.argmin(val_perf)
    else:
        i_best = np.argmax(val_perf)
    print("Best model selected, with")
    print("hyperparameters:        ", hparams[i_best])
    return i_best
    
def model_evaluation(model, X_train, Y_train, X_val, Y_val, X_test, Y_test, metric):   
    """
    Evaluate the selected model
    
    Train on the entire training/validation dataset
    Evaluate performance on both training/validation dataset and test dataset
    """
    # Take the whole training/validation dataset
    X_train_val = np.concatenate((X_train, X_val))
    Y_train_val = np.concatenate((Y_train, Y_val))
    
    # Train on the entire training/validation dataset
    model.fit(X_train_val, Y_train_val)
    
    # Evaluate performance
    train_val_perf = evaluate(model, X_train_val, Y_train_val, metric)
    test_perf = evaluate(model, X_test, Y_test, metric)
    print("Selected model performance:")
    print("Training (incl val):   %.4f" %train_val_perf)
    print("Test:                  %.4f" %test_perf) 
    return model, train_val_perf, test_perf

def basic_pipeline(
    X, Y, hparams, performance, 
    test_ratio=0.2, val_ratio=0.2, seed=264
):
    """
    Typical simple machine learning pipeline
    """

    # i. and ii.: Split dataset into a train, validation and test set
    X_train, Y_train, X_val, Y_val, X_test, Y_test = split_dataset(
        X, Y, test_ratio=test_ratio, val_ratio=val_ratio, seed=seed)
    
    train_perfs = []
    val_perfs = []
    models = []
    # iii. Loop over sets of hyperparameters:
    for hparam in hparams:
        print("\nUsing hyper-parameters:", hparam)
        
        # Instantiate model with current set of hyperparameter
        model, X_train_poly, X_val_poly, X_test_poly = polynomial_model(
            hparam['degree'], hparam["regularization"], X_train, X_val, X_test
        )
        
        # Train model
        model.fit(X_train_poly, Y_train)
        
        # Compute train and validation loss
        train_perf = evaluate(model, X_train_poly, Y_train, performance["metric"])
        val_perf = evaluate(model, X_val_poly, Y_val, performance["metric"])
        print("Training MSE:    %.4f" %train_perf)
        print("Validation MSE:  %.4f" %val_perf)
        
        # Store info about this model
        models.append(model)
        train_perfs.append(train_perf)
        val_perfs.append(val_perf)
        
    # Model selection
    i_best = model_selection(hparams, val_perfs, performance["minimize"])
    
    # Instantiate a model with the selected parameters
    best_params = hparams[i_best]
    best_model, X_train_poly, X_val_poly, X_test_poly = polynomial_model(
        best_params['degree'], best_params["regularization"], 
        X_train, X_val, X_test
    )
    
    # Evaluate the selected model
    best_model, train_val_perf, test_perf = model_evaluation(
        best_model, X_train_poly, Y_train, X_val_poly, Y_val, X_test_poly, Y_test, 
        performance["metric"]
    ) 
    return models, i_best

### Question 4: Machine learning pipeline with KFold cross-validation

Here we re-use some of the building blocks defined above and complement them in order to define a simple machine learning pipeline with KFold cross-validation 

In [ ]:
def KFold_split(X, Y, k=5, test_ratio=0.2, seed=264):
    """
    Split dataset into a test dataset and train/val kfolds
    """
    # Extract test set from entire dataset
    X_train_val, X_test, Y_train_val, Y_test = train_test_split(X, Y, test_size=test_ratio, shuffle=True, random_state=seed)
    
    # Create train/validation kfolds splitter
    KFold_splitter = KFold(n_splits=k, shuffle=True, random_state=seed)
    X_train_folds = []
    X_val_folds = []
    Y_train_folds = []
    Y_val_folds = []
    
    # Split train_val dataset into folds
    for (kth_fold_train_idxs, kth_fold_val_idxs) in KFold_splitter.split(X_train_val, Y_train_val):
        X_train_folds.append(X_train_val[kth_fold_train_idxs])
        X_val_folds.append(X_train_val[kth_fold_val_idxs])
        Y_train_folds.append(Y_train_val[kth_fold_train_idxs])
        Y_val_folds.append(Y_train_val[kth_fold_val_idxs])
        
    print("Training dataset size:   ", len(X_train_folds[0]))
    print("Validation dataset size: ", len(X_val_folds[0]))
    print("Test dataset size:       ", len(X_test))
    return X_train_folds, Y_train_folds, X_val_folds, Y_val_folds, X_test, Y_test

def pipeline_with_KFold(
    X, Y, hparams, performance, 
    k=5, test_ratio=0.2, seed=264
):
    # i. and ii.: Split dataset into a train, validation and test set
    X_train_folds, Y_train_folds, X_val_folds, Y_val_folds, X_test, Y_test = KFold_split(
        X, Y, k, test_ratio=test_ratio, seed=seed
    )
    
    train_mean_perfs = []
    val_mean_perfs = []
    models = []
    
    # iii. Loop over sets of hyperparameters:
    for hparam in hparams:
        print("\nUsing hyper-parameters:", hparam)
        
        train_perfs = []
        val_perfs = []
        
        # iv. Extra loop for the cross validation
        for X_train, X_val, Y_train, Y_val in zip(X_train_folds, X_val_folds, Y_train_folds, Y_val_folds):
        
            # v. Instantiate model with current set of hyperparameter
            model, X_train_poly, X_val_poly, X_test_poly = polynomial_model(
                hparam['degree'], hparam["regularization"], X_train, X_val, X_test
            )
            # Train model
            model.fit(X_train_poly, Y_train)
        
            # vi. Compute train and validation loss
            train_perf = evaluate(model, X_train_poly, Y_train, performance["metric"])
            val_perf = evaluate(model, X_val_poly, Y_val, performance["metric"])
            
            train_perfs.append(train_perf)
            val_perfs.append(val_perf)
            
        # vii. Compute mean performance for this set of hyperparameters
        train_mean_perf = np.mean(train_perfs)
        val_mean_perf = np.mean(val_perfs)
        print("Training mean MSE:    %.4f" %train_mean_perf)
        print("Validation mean MSE: %.4f" %val_mean_perf)
        
        # Store info about this set of hyperparameters
        train_mean_perfs.append(train_mean_perf)
        val_mean_perfs.append(val_mean_perf)
        
        # The model trained with the last fold will represent all other
        # models trained with this set of hyperparameters but on other folds
        models.append(model)
        
    # viii. Model selection
    i_best = model_selection( hparams, val_mean_perfs, performance["minimize"])
    
    # ix. Instantiate a model with the selected parameters
    best_params = hparams[i_best]
    best_model, X_train_poly, X_val_poly, X_test_poly = polynomial_model(
        best_params['degree'], best_params["regularization"], 
        X_train_folds[0], X_val_folds[0], X_test
    )
    
    # Evaluate the selected model
    best_model, train_val_perf, test_perf = model_evaluation(
        best_model, 
        X_train_poly, Y_train_folds[0], 
        X_val_poly, Y_val_folds[0], 
        X_test_poly, Y_test, 
        performance["metric"]
    ) 
    return models, i_best

In [ ]:
perf={"metric" : mean_squared_error, "minimize" : True}

# Create the list of hyper-parameters instances:
hyper_parameters = [
    {"degree": 1, "regularization": 0},
    {"degree": 2, "regularization": 0},
    {"degree": 3, "regularization": 0},
    {"degree": 4, "regularization": 0},
    {"degree": 1, "regularization": 0.001},
    {"degree": 2, "regularization": 0.001},
    {"degree": 3, "regularization": 0.001},
    {"degree": 4, "regularization": 0.001},
    {"degree": 1, "regularization": 0.01},
    {"degree": 2, "regularization": 0.01},
    {"degree": 3, "regularization": 0.01},
    {"degree": 4, "regularization": 0.01},
    {"degree": 1, "regularization": 0.1},
    {"degree": 2, "regularization": 0.1},
    {"degree": 3, "regularization": 0.1},
    {"degree": 4, "regularization": 0.1}]

In [ ]:
# Select model with regular validation:
models, i_best = basic_pipeline(X, Y, hyper_parameters, perf)

In [ ]:
# Select model with KFold cross-validation:
models, i_best = pipeline_with_KFold(X, Y, hyper_parameters, perf)

### Question 5: Effect of regularization

A singular matrix warning indicates that the matrix containing the data has columns that are approximately linearly dependant, thus the matrix becomes close to singular. 

Increasing the regularization hyper-parameter will eventually get rid of this warning. Indeed, recall that the Ridge model solves 
$$\underset{w}{\min} \|y-Xw\|_2^2 + \alpha\|w\|_2^2$$
Then, setting the gradient to 0 yields a unique solution: 
$$w_{sol} = (X^tX + \alpha I)^{-1} X^ty$$
The higher the $\alpha$ in $X^tX + \alpha I$, the more dominant the identity matrix term becomes compared to the potentially singular matrix term $X^tX$. When the identity matrix is dominant enough the matrix $X^tX + \alpha I$ becomes invertible i.e. non-singular. 

Intuitively, the penalization term $\alpha\|w\|_2^2$ forces the model to shrink the solution $w_{sol}$ toward $0$ (the bigger $\alpha$ is, the stronger the shrinkage), which lowers the impact from irrelevant variables in the model and makes the model more resilient to columns (features) colinearity in the data matrix.

In the context of this exercice, with a Ridge model of degree $3$ we may have to push the regularization hyper-parameter at a very high value ($>100$) to get rid of the singular matrix warning, and we then may get an ill-conditioned matrix warning instead. To get rid of this new warning, the regularization hyper-parameter should be set even higher.

Notice that if you run the model selections several time, the cross-validation tends to be more stable than the regular validation in the sense that certain hyperparameters instances are selected quite often with cross-validation.